# 環境設定

In [ ]:
# 下載講師自製的 HappyML
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

In [ ]:
# 載入必要套件
import pandas as pd
from sklearn.datasets import load_iris

import HappyML.preprocessor as pp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# 檢查是否有可用的 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 資料集前處理

In [ ]:
# 載入資料集
dataset = load_iris()

In [ ]:
# 切分自變數與應變數
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
Y = pd.DataFrame(dataset.target, columns=['Iris_Type'])
Y_name = dataset.target_names.tolist()

# 可試著印出 X, Y, Y_name 驗證結果
print(X, Y)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]      Iris_Type
0            0
1         

In [ ]:
# 切分訓練集、測試集
X_train, X_test, Y_train, Y_test = pp.split_train_test(x_ary=X, y_ary=Y)
print(X_train, X_test, Y_train, Y_test)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
113                5.7               2.5                5.0               2.0
45                 4.8               3.0                1.4               0.3
64                 5.6               2.9                3.6               1.3
147                6.5               3.0                5.2               2.0
119                6.0               2.2                5.0               1.5
..                 ...               ...                ...               ...
91                 6.1               3.0                4.6               1.4
49                 5.0               3.3                1.4               0.2
110                6.5               3.2                5.1               2.0
18                 5.7               3.8                1.7               0.3
88                 5.6               3.0                4.1               1.3

[112 rows x 4 columns]      sepal length (cm)  sepal width (cm)

In [ ]:
# 特徵縮放
X_train, X_test = pp.feature_scaling(fit_ary=X_train, transform_arys=(X_train, X_test))

# 可試著印出 X_train, X_test, Y_train, Y_test 驗證結果
print(X_train, X_test)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
113          -0.157339         -1.396168           0.741024          1.088726
45           -1.214654         -0.222213          -1.275041         -1.130690
64           -0.274818         -0.457004          -0.043001          0.174849
147           0.782497         -0.222213           0.853028          1.088726
119           0.195100         -2.100541           0.741024          0.435957
..                 ...               ...                ...               ...
91            0.312579         -0.222213           0.517017          0.305403
49           -0.979695          0.482160          -1.275041         -1.261244
110           0.782497          0.247369           0.797026          1.088726
18           -0.157339          1.656115          -1.107036         -1.130690
88           -0.274818         -0.222213           0.237008          0.174849

[112 rows x 4 columns]      sepal length (cm)  sepal width (cm)

In [ ]:
# 將資料轉換成 PyTorch 張量
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)

# 可試著印出 X_train_tensor, X_test_tensor, Y_train_tensor, Y_test_tensor 驗證
print(X_train_tensor, X_test_tensor, Y_train_tensor, Y_test_tensor)

tensor([[-0.1573, -1.3962,  0.7410,  1.0887],
        [-1.2147, -0.2222, -1.2750, -1.1307],
        [-0.2748, -0.4570, -0.0430,  0.1748],
        [ 0.7825, -0.2222,  0.8530,  1.0887],
        [ 0.1951, -2.1005,  0.7410,  0.4360],
        [ 2.4272,  1.6561,  1.5250,  1.0887],
        [ 1.6049,  1.1865,  1.3570,  1.7415],
        [ 0.4301,  0.7170,  0.9650,  1.4804],
        [ 1.8398, -0.6918,  1.3570,  0.9582],
        [-0.5098,  0.7170, -1.1070, -1.2612],
        [-1.0972,  0.0126, -1.2190, -1.2612],
        [ 0.5475, -0.6918,  0.7970,  0.4360],
        [-0.9797,  0.9517, -1.1630, -0.7390],
        [ 1.2524,  0.0126,  0.7970,  1.4804],
        [-0.0399, -0.9266,  0.7970,  0.9582],
        [ 2.1923, -0.2222,  1.3570,  1.4804],
        [ 0.0776, -0.2222,  0.2930,  0.4360],
        [ 0.6650, -0.9266,  0.9090,  0.9582],
        [ 1.1349, -0.2222,  1.0210,  1.2193],
        [-0.5098, -0.2222,  0.4610,  0.4360],
        [-0.5098,  0.7170, -1.2190, -1.0001],
        [-1.4496,  0.0126, -1.2190

# 模型定義

## 定義代表模型的類別

In [ ]:
class IrisModel(nn.Module):

  # 定義神經網路每層架構
  def __init__(self):
    super(IrisModel, self).__init__()

    # 先定義每個神經層
    self.fc1 = nn.Linear(4, 16)
    self.fc2 = nn.Linear(16, 16)
    self.fc3 = nn.Linear(16, 3)

    # 接著初始化每個神經層的權重
    init.xavier_normal_(self.fc1.weight)
    init.xavier_normal_(self.fc2.weight)
    init.xavier_normal_(self.fc3.weight)

  # 定義輸入值 x 如何一路計算到輸出值（正向傳播）
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)

    return x

## 定義優化器與損失函數

In [ ]:
# 將模型實體化
model = IrisModel().to(device)

# 定義損失函數
criterion = nn.CrossEntropyLoss()

# 定義優化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型

In [ ]:
# 將資料集切割成數個 batch
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 開始訓練
epochs = 100

for epoch in range(epochs):
  # 將模型設定為訓練模式
  model.train()

  # 儲存猜對的數量 ＆ 完整數量
  correct = 0
  total = 0

  # 取出一個 Batch 開始訓練
  for X_batch, Y_batch in train_loader:
    # 將 Batch 的資料轉換到 GPU 上
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    # 先把上一個 Batch 的梯度歸零
    optimizer.zero_grad()

    # 計算輸出值
    Y_pred = model(X_batch)
    # 計算損失值
    loss = criterion(Y_pred, Y_batch.squeeze())
    # 根據損失值求微分找損失極小的權重（反向傳播）
    loss.backward()
    # 將求出來的權重實際更新上去
    optimizer.step()

    # 計算第 1 軸的每一列，最大值之索引為何
    _, predicted = torch.max(Y_pred.data, 1)
    # 將這一批次有幾筆資料加入到 total 中
    total += Y_batch.size(0)
    # 計算猜對的數量（.item() 會協助取得純量）
    correct += (predicted == Y_batch.squeeze()).sum().item()

  # 計算每個 epoch 的準確率
  accuracy = correct / total
  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Acc: {accuracy:.4f}')

Epoch 1/100, Loss: 1.1438, Acc: 0.1786
Epoch 2/100, Loss: 1.0637, Acc: 0.2232
Epoch 3/100, Loss: 1.0450, Acc: 0.3304
Epoch 4/100, Loss: 1.0285, Acc: 0.5089
Epoch 5/100, Loss: 0.9829, Acc: 0.6875
Epoch 6/100, Loss: 0.9602, Acc: 0.7768
Epoch 7/100, Loss: 0.9658, Acc: 0.8125
Epoch 8/100, Loss: 0.9203, Acc: 0.8393
Epoch 9/100, Loss: 0.8591, Acc: 0.8571
Epoch 10/100, Loss: 0.9191, Acc: 0.8750
Epoch 11/100, Loss: 0.8382, Acc: 0.9196
Epoch 12/100, Loss: 0.8267, Acc: 0.9643
Epoch 13/100, Loss: 0.7746, Acc: 0.9643
Epoch 14/100, Loss: 0.7951, Acc: 0.9643
Epoch 15/100, Loss: 0.7422, Acc: 0.9464
Epoch 16/100, Loss: 0.6602, Acc: 0.9286
Epoch 17/100, Loss: 0.7226, Acc: 0.9286
Epoch 18/100, Loss: 0.6190, Acc: 0.9107
Epoch 19/100, Loss: 0.7851, Acc: 0.8839
Epoch 20/100, Loss: 0.6686, Acc: 0.8839
Epoch 21/100, Loss: 0.5960, Acc: 0.8750
Epoch 22/100, Loss: 0.6535, Acc: 0.8750
Epoch 23/100, Loss: 0.5678, Acc: 0.8750
Epoch 24/100, Loss: 0.5993, Acc: 0.8839
Epoch 25/100, Loss: 0.5714, Acc: 0.8839
Epoch 26/

# 評估模型

In [ ]:
# 將模型切換為評估模式
model.eval()

# 關閉 PyTorch 的梯度計算機制（Evaluation 時期不需要它）
with torch.no_grad():

  # 儲存猜對的數量 ＆ 完整數量
  correct = 0
  total = 0

  # 取出測試集的一個批次，開始測試
  for X_batch, Y_batch in test_loader:
    # 將 Batch 的資料轉換到 GPU 上
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    # 計算輸出值
    Y_pred = model(X_batch)

    # 找到每一列預測機率最高的數值與其索引，即模型的預測類別。
    _, predicted = torch.max(Y_pred.data, 1)
    # 將這一批次有幾筆資料加入到 total 中
    total += Y_batch.size(0)
    # 計算猜對的數量（.item() 會協助取得純量）
    correct += (predicted == Y_batch.squeeze()).sum().item()

  # 計算每個 epoch 的準確率
  print(f'Test Accuracy: {correct / total:.4f}')

Test Accuracy: 0.8684
